<img src="Mars.jpg" alt="The first colored picture sent by Perseverance" width="600"/>

## Familiarize yourself with Gaussian Mixtures

In this lab we will try to feel comfortable with the Expectation Maximization algorithm and its use to fit Gaussian mixture models. To this aim we will start by writing our own EM algorithm for a mixture of univariate Gaussian distributions. We will then complexify our setting by dealing with the multivariate case. We will finally have fun with an application to image segmentation on the first colored picture of Mars sent recently by *Perseverance*.

*Remark: Although we will use the GaussianMixture function of Python for this lab, the current version is not as powerfull as (many) R alternative such as packages *mclust* or *Rmixmod*. For instance, such R packages allows for much more covariance structure configurations than the GaussianMixture. If you want to dive seriously into the Gaussian mixture model worlds, these software are top notch!*

### Univariate case

We consider the following mixture model
$$f(y) = \sum_{k=1}^K \tau_k \varphi(y; \mu_k, \sigma^2_k), \qquad k \geq 1,$$
where $\varphi(\cdot, \mu, \sigma^2)$ denotes the p.d.f. of a normal distribution with mean $\mu$ and variance $\sigma^2$ and $\tau_k$ are the cluster membership probabilities, i.e., $\tau_k \geq 0$ and $\sum_{k} \tau_k = 1$.

In [1]:
import numpy as np

#### Question 1:

Write a function that simulates from this mixture model. Play a bit with your brand new function to see how changing the parameters impacts the structure of the observations.

In [27]:
np.random.normal(scale=np.array([8,1]))

array([-1.49403083, -0.95582518])

In [30]:
def gauss(yi, mu, sigma) :
    return(1/(sigma * np.sqrt(2 * np.pi)) *
               np.exp( - (yi - mu)**2 / (2 * sigma**2) ))

In [31]:
gauss(point[0], mu[0], sigma[0])

array([0.02195823])

In [33]:
sum(mu)

27

In [108]:
tau = [0.2, 0.2, 0.2, 0.2, 0.2]
mu = [1, 5, 13, 6, 2]
sigma = [8, 1, 0.5, 3, 10]
n = 50

point = []
classe = []
for i in range(n) : 
    nb = np.random.choice([0, 1, 2 ,3, 4], p=tau)
    classe.append(nb)
    point.append(np.random.normal(mu[nb], sigma[nb], 1))

point, classe

([array([2.22307849]),
  array([13.70142218]),
  array([-5.8675324]),
  array([12.64497577]),
  array([6.75812897]),
  array([3.88426731]),
  array([3.89525342]),
  array([13.29066445]),
  array([19.12010821]),
  array([12.5651107]),
  array([4.70728456]),
  array([4.87132509]),
  array([7.67757138]),
  array([7.26747298]),
  array([13.24779858]),
  array([5.01534067]),
  array([10.32339551]),
  array([5.65860369]),
  array([8.74373246]),
  array([12.67127004]),
  array([12.65655019]),
  array([6.58766844]),
  array([5.06409995]),
  array([-8.36241023]),
  array([9.17541255]),
  array([2.54559172]),
  array([3.02714109]),
  array([13.89461849]),
  array([12.93181576]),
  array([2.72472237]),
  array([10.55834219]),
  array([13.37327933]),
  array([5.31481558]),
  array([13.69205034]),
  array([9.42364766]),
  array([4.57545784]),
  array([12.74224543]),
  array([-1.96185714]),
  array([-13.83509816]),
  array([7.22616782]),
  array([-2.04739981]),
  array([5.83262385]),
  array([4.7084

#### Question 2:

Write a pseudo code for an EM algorithm on this mixture model and, next, implement it.

In [109]:
I = 0
mu = [1, 1, 1, 1,1]
tau = [0.2, 0.2, 0.2, 0.2, 0.2]
sigma = [1, 1, 1, 1, 1]

while I<4 : 
    
    for g in range(5) :
        tau_cond = []
        for i in range(len(point)) :
            somme = 0
            for k in range(5) : 
                somme = somme + tau[k]*gauss(point[i], mu[k], sigma[k])
            tau_cond.append(tau[g]*gauss(point[i], mu[g], sigma[g])/somme)
        print(I)
        print(tau_cond)
        tau[g] = sum(tau_cond)/len(point)
        
        multi = 0
        for i in range(len(point)):
            multi = multi+tau_cond[i]*point[i]
        mu[g] = multi/sum(tau_cond)
        
        cov = 0
        for i in range(len(point)):
            cov = cov+tau_cond[i]*(point[i]-mu[g])**2
        sigma[g] = cov/sum(tau_cond)
    print("AAAAAAAAAAAA")
    print(tau)
    I = I+1


        
    

0
[array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2]), array([0.2])]
0
[array([0.24702233]), array([4.10511631e-34]), array([2.60707979e-09]), array([1.57377372e-28]), array([2.75140858e-06]), array([0.1829403]), array([0.18137018]), array([6.94796407e-32]), array([2.28598151e-70]), array([3.97516886e-28]), array([0.03829217]), array([0.02213225]), array([9.053545

In [110]:
tau

[array([0.05918437]),
 array([0.07188149]),
 array([0.15754609]),
 array([0.39600934]),
 array([0.56676901])]

In [111]:
sigma

[array([75.16806525]),
 array([76.66528877]),
 array([77.30081275]),
 array([71.44180498]),
 array([12.05722521])]

In [71]:
g = 0
for i in range(len(point)) :
            somme = 0
            for g in range(5) : 
                somme = somme + tau[g]*gauss(point[i], mu[g], sigma[g])
            tau_cond.append(tau[g]*gauss(point[i], mu[g], sigma[g])/somme)
a = sum(tau_cond)/len(point)
a

array([0.30922452])

#### Question 3:

Test your EM algorithm on simulated data and show the evolution of the Gaussian mixture estimates as the number of iterations increases.

In [12]:
## Code goes here
## %load solutions/question3.py

#### Question 4:

Modify your EM algorithm so that it also outputs the conditional membership probabilities

$$ t_g(y) = \Pr(Z = g \mid Y = y), \qquad y \in \mathcal{Y}, \quad g \in \{1, \ldots, G\},$$

and, based on your simulated data, predict the class of each observation using maximum a posteriori.

You may consider plotting the data with colors depending on the estimated component, the fitted mixture density and each individuals gaussian densities.

In [15]:
## Code goes here
## %load solutions/question4.py

#### Question 5:

Write a function that predict the class of new observations.

In [18]:
## Code goes here
## %load solutions/question5.py

#### Question 6:

Have a look at the [GaussianMixture](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html) function of the scikitlearn library and learn how to use it. 

Use it on your simulated dataset and compare the output from that of your own function. You may want to:
  + redo the above graphics using the output of GaussianMixture;
  + use the [confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) function.
  
Are you happy?

In [21]:
## Code goes here
## %load solutions/question6.py

### Going multivariate

#### Question 7:

Have a look at the [make_blobs](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html) function which generate mixture from (very specific) Gaussian distributions. Use this function to generate a sample of size $n=100$ from a Gaussian mixture in $\mathbb{R}^2$ with $G=4$ components.

In [24]:
## Code goes here
## %load solutions/question7.py

#### Question 8:

Use the GaussianMixture function to estimate a Gaussian mixture model on this sample. Give the mathematical details of the model you fit. Give a plot where you show the prediction class in $[-12, 12]^2$ and another plot where you plot the uncertainties in classification. 

In [33]:
## Code goes here
## %load solutions/question8.py

#### Question 9:

Perform model selection by varying the number of mixture components and the covariance structure. Compare prediction from the top two competitive models.

In [36]:
## Code goes here
## %load solutions/question9.py

### Just for fun Image segmentation

Here we will do a small application where we want to perform image segmentation using Gaussian mixture models. Here is a typical roadmap:
  - First get the [first colored image of Mars sent by *Perseverance*](https://mars.nasa.gov/system/resources/detail_files/25612_PIA24430-panorama-1200.jpg) and import it.
  - Then fit a suitable Gaussian mixture model. In doing so, you may want to train your model on a (random) sub-sample of this image.
  - Plot the "segmented" image where you first plot using "non-sense" color palette, i.e., color palette is $\{1, \ldots, G\}$.
  - Refine your color platte by taking it to be $\{\mu_1, \ldots, \mu_G\}$.

In [35]:
## Code goes here
## %load solutions/justForFun.py